In [1]:
import json
from pathlib import Path
from functools import reduce

import pandas as pd

from IPython.display import display

In [2]:
ARTICLES_DIR = "../../data/test/user_study/articles/"
STOCKS_PATH = "../../data/processed/stocks/revolut.2021-07-05.complete.stocks.jsonl"
RATINGS_PATH = "../../data/temp/dataclips_gdnjjptpwsyunzmhlqhqsodmmqda.json"

###  Sorting out paths

In [3]:
def key(path):
    stem = path.stem
    if stem.startswith("oos"):
        return int(stem[-1])
    else:
        return int(stem)

In [4]:
# Sorting out paths
article_paths = sorted(list(Path(ARTICLES_DIR).glob("*.json")), key=key)
article_paths

[PosixPath('../../data/test/user_study/articles/0.json'),
 PosixPath('../../data/test/user_study/articles/oos_0.json'),
 PosixPath('../../data/test/user_study/articles/1.json'),
 PosixPath('../../data/test/user_study/articles/oos_1.json'),
 PosixPath('../../data/test/user_study/articles/oos_2.json'),
 PosixPath('../../data/test/user_study/articles/3.json'),
 PosixPath('../../data/test/user_study/articles/oos_3.json'),
 PosixPath('../../data/test/user_study/articles/4.json'),
 PosixPath('../../data/test/user_study/articles/oos_4.json'),
 PosixPath('../../data/test/user_study/articles/oos_5.json'),
 PosixPath('../../data/test/user_study/articles/5.json'),
 PosixPath('../../data/test/user_study/articles/oos_6.json'),
 PosixPath('../../data/test/user_study/articles/6.json'),
 PosixPath('../../data/test/user_study/articles/oos_7.json'),
 PosixPath('../../data/test/user_study/articles/7.json'),
 PosixPath('../../data/test/user_study/articles/8.json'),
 PosixPath('../../data/test/user_study/a

### Inverting articles dict for relevant articles

In [5]:
def read_json(path):
    with open(path, "r") as fp:
        return json.load(fp)

In [6]:
articles = dict()
stocks = dict()

for path in article_paths:
    
    article = read_json(path)
    articles[article["doc_id"]] = article
    
    for stock in article["stock_matches"]:
        ticker = stock["ticker_symbol"]
        score = stock.pop("score")
        
        if ticker not in stocks.keys():    
            stocks[ticker] = stock
            stocks[ticker]["relevant_articles"] = list()
            
        article_match = {k:v for k,v in article.items() if k in ["doc_id", "title", "url", "summary"]}
        article_match["score"] = score
        stocks[ticker]["relevant_articles"].append(article_match)
        
        

In [7]:
for key, values in stocks.items():
    print("{}\t{}".format(key,len(values["relevant_articles"])))

LUMN	1
VG	1
CMCSA	1
CABO	1
VIACA	1
ROKU	1
ATUS	1
VZ	1
CHTR	1
GRPN	1
SPCE	2
LMT	2
WORK	2
KTOS	1
LI	4
PTRA	3
CVNA	1
NRG	5
SO	2
DTE	1
ARVL	1
F	1
GLW	1
AEP	2
CNP	1
TW	2
CME	1
HES	3
NMR	2
MPC	2
AR	2
NOV	2
KKR	2
YPF	2
EQNR	3
BEP	2
RIO	3
TECK	3
COP	2
CVE	2
ESI	1
SRE	2
BHP	2
PCG	1
XPEV	4
NIO	3
TSLA	2
BLK	1
GME	1
NDAQ	5
BABA	5
GS	1
COIN	1
MS	1
UBS	1
IBKR	1
FB	2
TM	2
XLNX	5
HIMX	3
DIDI	3
NAVI	1
BNTX	3
NVAX	2
GSK	1
TEVA	1
SNY	1
DXC	1
AZN	2
ABBV	1
ARCT	1
MRNA	3
BUD	1
ABEV	2
KDP	1
KHC	1
STZ	1
HPE	1
YETI	1
MAR	2
YUMC	1
VFC	2
ARNC	1
HAL	1
OLN	1
H	2
TEAM	1
RCL	2
NCLH	2
CCL	3
NOC	1
HEI	1
SAVE	1
JBLU	1
AMC	1
LEVI	1
RL	2
REAL	1
URBN	1
GM	1
IP	1
BP	1
GGB	1
EDU	3
TME	3
TAL	1
CHKP	1
QCOM	2
NTAP	2
ZM	1
MCFE	1
WDC	3
VMW	1
PANW	1
PINS	1
FCEL	2
RSG	1
PLUG	1
CHPT	1
AMD	4
ZS	2
YY	5
STX	2
DQ	1
TSM	4
SWKS	2
NVDA	2
INTC	2
GPRO	1
ANET	1
MCHP	2
SPWR	1
TXN	1
CDNS	1
ON	1
ZNGA	1
ATVI	1
SPOT	1
AAPL	1
MA	1
ADBE	1
RBLX	1
PYPL	1
VLO	1
FANG	1
CLDR	2
BIP	2
RUN	1
PBR	1
SCCO	3
PBF	2
VALE	3
SID	3
OVV	2
SOFI	2
MDB	1
BSBR	3
SSNC	2


In [8]:
print(len(stocks))

246


In [9]:
test_articles_top_tickers = reduce(set.union, [set(article["ticker_top"]) for article in articles.values() 
                                               if "ticker_top" in article.keys()])

missed_top_tickers = test_articles_top_tickers - set(stocks.keys())
print("Top tickers that are not in any article", missed_top_tickers)

Top tickers that are not in any article {'PFE', 'FCX', 'XOM', 'T', 'X', 'BBD', 'CVX'}


### Getting top stocks that were not in any article

In [10]:
def read_jsonl(path, missed_tickers):
    with open(path, "r") as fp:
        for i, line in enumerate(fp):
            stock = json.loads(line)
            if stock["ticker_symbol"] in missed_tickers:
                stock_ = dict()
                stock_["index"] = i
                stock_.update({k:v for k,v in stock.items() 
                               if k in ["stock_name", "ticker_symbol", "sector", "industry", "comment"]})
                stock_["relevant_articles"] = []
                yield {stock_["ticker_symbol"]: stock_}

In [11]:
for stock_ in read_jsonl(STOCKS_PATH, missed_tickers=missed_top_tickers):
    stocks.update(stock_)
    
print(len(stocks))

253


### Setting aside stocks with at least two relavant articles

In [12]:
stocks_df = pd.DataFrame.from_dict(stocks, orient="index")
stocks_df["article_count"] = stocks_df.relevant_articles.apply(len)

set_aside_stocks = (stocks_df[(stocks_df.article_count > 2) | 
                              (stocks_df.ticker_symbol.isin(test_articles_top_tickers))]
                        .to_dict(orient="index"))

print(len(set_aside_stocks))
print(set_aside_stocks.keys())

56
dict_keys(['LI', 'PTRA', 'NRG', 'F', 'HES', 'EQNR', 'RIO', 'TECK', 'XPEV', 'NIO', 'TSLA', 'NDAQ', 'BABA', 'XLNX', 'HIMX', 'DIDI', 'BNTX', 'MRNA', 'ABEV', 'NCLH', 'CCL', 'AMC', 'EDU', 'TME', 'WDC', 'PLUG', 'AMD', 'YY', 'TSM', 'NVDA', 'INTC', 'AAPL', 'PBR', 'SCCO', 'VALE', 'SID', 'SOFI', 'BSBR', 'C', 'ITUB', 'EVR', 'ET', 'KSS', 'M', 'JD', 'VIPS', 'IQ', 'UBER', 'AAL', 'T', 'BBD', 'CVX', 'XOM', 'FCX', 'PFE', 'X'])


### Adding in stocks that have at least one evaluation rating

In [13]:
rated_stocks = read_json(RATINGS_PATH)
rated_stocks = [rated_stocks["values"][i][1] for i in range(len(rated_stocks["values"]))]

In [14]:
rated_stocks

['HES',
 'AAPL',
 'BABA',
 'ABEV',
 'VIPS',
 'ATVI',
 'NIO',
 'NOV',
 'UBER',
 'DIDI',
 'AMD',
 'AMC',
 'AAL',
 'CCL',
 'C',
 'KHC',
 'T',
 'ET',
 'VFC',
 'ROKU',
 'KDP',
 'ESI',
 'BSBR',
 'KHC',
 'T',
 'X',
 'NVDA',
 'TSLA',
 'AMC',
 'KHC',
 'CVX',
 'PFE',
 'SPCE',
 'INTC',
 'BBD',
 'MRNA',
 'VALE',
 'TSM',
 'NDAQ',
 'BABA',
 'XOM',
 'F',
 'FCX',
 'IQ',
 'ITUB',
 'KSS']

### Adding rated stocks to set aside stocks list

In [15]:
missing_stocks = set(rated_stocks) - set(set_aside_stocks.keys())
missing_stocks

{'ATVI', 'ESI', 'KDP', 'KHC', 'NOV', 'ROKU', 'SPCE', 'VFC'}

In [16]:
for ticker in missing_stocks:
    set_aside_stocks.update(dict({ticker: stocks[ticker]}))

In [17]:
set_aside_stocks.keys()

dict_keys(['LI', 'PTRA', 'NRG', 'F', 'HES', 'EQNR', 'RIO', 'TECK', 'XPEV', 'NIO', 'TSLA', 'NDAQ', 'BABA', 'XLNX', 'HIMX', 'DIDI', 'BNTX', 'MRNA', 'ABEV', 'NCLH', 'CCL', 'AMC', 'EDU', 'TME', 'WDC', 'PLUG', 'AMD', 'YY', 'TSM', 'NVDA', 'INTC', 'AAPL', 'PBR', 'SCCO', 'VALE', 'SID', 'SOFI', 'BSBR', 'C', 'ITUB', 'EVR', 'ET', 'KSS', 'M', 'JD', 'VIPS', 'IQ', 'UBER', 'AAL', 'T', 'BBD', 'CVX', 'XOM', 'FCX', 'PFE', 'X', 'ROKU', 'VFC', 'KDP', 'ATVI', 'SPCE', 'NOV', 'KHC', 'ESI'])

In [18]:
len(set_aside_stocks)

64

### Choosing representative stocks from sectors and industries  that are not set aside yet

In [19]:
set_aside_sectors_industries = [(stock["sector"], stock["industry"]) for stock in set_aside_stocks.values()]
print(len(set_aside_sectors_industries))
print(len(set(set_aside_sectors_industries)))

64
37


In [20]:
sectors_industries = stocks_df.groupby(["sector", "industry"])
remaining_serctors_indsturies = set(sectors_industries.groups.keys()) - set(set_aside_sectors_industries)
remaining_serctors_indsturies

{('Commercial Services', 'Advertising/Marketing Services'),
 ('Commercial Services', 'Financial Publishing/Services'),
 ('Communications', 'Specialty Telecommunications'),
 ('Communications', 'Telecommunications Equipment'),
 ('Communications', 'Wireless Telecommunications'),
 ('Consumer Durables', 'Homebuilding'),
 ('Consumer Non-Durables', 'Household/Personal Care'),
 ('Consumer Services', 'Broadcasting'),
 ('Consumer Services', 'Cable/Satellite TV'),
 ('Consumer Services', 'Restaurants'),
 ('Electronic Technology', 'Computer Communications'),
 ('Electronic Technology', 'Computer Processing Hardware'),
 ('Electronic Technology', 'Electronic Equipment/Instruments'),
 ('Energy Minerals', 'Oil Refining/Marketing'),
 ('Finance', 'Investment Managers'),
 ('Finance', 'Life/Health Insurance'),
 ('Finance', 'Real Estate Development'),
 ('Finance', 'Real Estate Investment Trusts'),
 ('Health Technology', 'Medical Specialties'),
 ('Health Technology', 'Pharmaceuticals: Generic'),
 ('Industrial

In [21]:
unrepresented_sectors_industries = []

for key, group in sectors_industries:
    if key in remaining_serctors_indsturies:
        group = group.sort_values("article_count", ascending=False)
        print(key)
        display(group)
        top_match = group[group.article_count > 1].head(1)
        if top_match is not None:
            top_match.pop("article_count")
            set_aside_stocks.update(top_match.to_dict(orient="index"))
            unrepresented_sectors_industries.append(key)


('Commercial Services', 'Advertising/Marketing Services')


,index,stock_name,ticker_symbol,sector,industry,comment,relevant_articles,article_count
GRPN,326,Groupon,GRPN,Commercial Services,Advertising/Marketing Services,Groupon is an American global e-commerce marke...,"[{'doc_id': 0, 'title': 'AT&T CFO Pascal Desro...",1


('Commercial Services', 'Financial Publishing/Services')


,index,stock_name,ticker_symbol,sector,industry,comment,relevant_articles,article_count
MCO,476,Moody’s,MCO,Commercial Services,Financial Publishing/Services,"Moody's Corporation, often referred to as Mood...","[{'doc_id': 23, 'title': 'Citigroup floats $23...",1


('Communications', 'Specialty Telecommunications')


...

,index,stock_name,ticker_symbol,sector,industry,comment,relevant_articles,article_count
UNP,706,Union Pacific,UNP,Transportation,Railroads,The Union Pacific Corporation (Union Pacific) ...,"[{'doc_id': 34, 'title': 'US Steel Output Spik...",1


('Utilities', 'Alternative Power Generation')


,index,stock_name,ticker_symbol,sector,industry,comment,relevant_articles,article_count
BIP,121,Brookfield Infrastructure Partners,BIP,Utilities,Alternative Power Generation,Brookfield Infrastructure Partners L.P. is a p...,"[{'doc_id': 19, 'title': 'Why Chevron And Exxo...",2
RUN,653,Sunrun,RUN,Utilities,Alternative Power Generation,Sunrun Inc. is an American provider of residen...,"[{'doc_id': 19, 'title': 'Why Chevron And Exxo...",1


('Utilities', 'Gas Distributors')


,index,stock_name,ticker_symbol,sector,industry,comment,relevant_articles,article_count
SRE,611,Sempra Energy,SRE,Utilities,Gas Distributors,Sempra is a North American energy infrastructu...,"[{'doc_id': 'oos_2', 'title': 'COP26 Climate S...",2


In [22]:
print(len(set_aside_stocks))
set_aside_stocks.keys()

75


dict_keys(['LI', 'PTRA', 'NRG', 'F', 'HES', 'EQNR', 'RIO', 'TECK', 'XPEV', 'NIO', 'TSLA', 'NDAQ', 'BABA', 'XLNX', 'HIMX', 'DIDI', 'BNTX', 'MRNA', 'ABEV', 'NCLH', 'CCL', 'AMC', 'EDU', 'TME', 'WDC', 'PLUG', 'AMD', 'YY', 'TSM', 'NVDA', 'INTC', 'AAPL', 'PBR', 'SCCO', 'VALE', 'SID', 'SOFI', 'BSBR', 'C', 'ITUB', 'EVR', 'ET', 'KSS', 'M', 'JD', 'VIPS', 'IQ', 'UBER', 'AAL', 'T', 'BBD', 'CVX', 'XOM', 'FCX', 'PFE', 'X', 'ROKU', 'VFC', 'KDP', 'ATVI', 'SPCE', 'NOV', 'KHC', 'ESI', 'CHL', 'COTY', 'FCEL', 'MPC', 'KKR', 'INO', 'IAG', 'RL', 'SSNC', 'BIP', 'SRE'])

In [23]:
for pairing in unrepresented_sectors_industries:
    print(pairing)

('Commercial Services', 'Advertising/Marketing Services')
('Commercial Services', 'Financial Publishing/Services')
('Communications', 'Specialty Telecommunications')
('Communications', 'Telecommunications Equipment')
('Communications', 'Wireless Telecommunications')
('Consumer Durables', 'Homebuilding')
('Consumer Non-Durables', 'Household/Personal Care')
('Consumer Services', 'Broadcasting')
('Consumer Services', 'Cable/Satellite TV')
('Consumer Services', 'Restaurants')
('Electronic Technology', 'Computer Communications')
('Electronic Technology', 'Computer Processing Hardware')
('Electronic Technology', 'Electronic Equipment/Instruments')
('Energy Minerals', 'Oil Refining/Marketing')
('Finance', 'Investment Managers')
('Finance', 'Life/Health Insurance')
('Finance', 'Real Estate Development')
('Finance', 'Real Estate Investment Trusts')
('Health Technology', 'Medical Specialties')
('Health Technology', 'Pharmaceuticals: Generic')
('Industrial Services', 'Environmental Services')
('N

### Adding irrelevant articles

In [24]:
set_aside_stocks.keys()

dict_keys(['LI', 'PTRA', 'NRG', 'F', 'HES', 'EQNR', 'RIO', 'TECK', 'XPEV', 'NIO', 'TSLA', 'NDAQ', 'BABA', 'XLNX', 'HIMX', 'DIDI', 'BNTX', 'MRNA', 'ABEV', 'NCLH', 'CCL', 'AMC', 'EDU', 'TME', 'WDC', 'PLUG', 'AMD', 'YY', 'TSM', 'NVDA', 'INTC', 'AAPL', 'PBR', 'SCCO', 'VALE', 'SID', 'SOFI', 'BSBR', 'C', 'ITUB', 'EVR', 'ET', 'KSS', 'M', 'JD', 'VIPS', 'IQ', 'UBER', 'AAL', 'T', 'BBD', 'CVX', 'XOM', 'FCX', 'PFE', 'X', 'ROKU', 'VFC', 'KDP', 'ATVI', 'SPCE', 'NOV', 'KHC', 'ESI', 'CHL', 'COTY', 'FCEL', 'MPC', 'KKR', 'INO', 'IAG', 'RL', 'SSNC', 'BIP', 'SRE'])

In [25]:
for _, stock in set_aside_stocks.items():
    stock["irrelevant_articles"] = list()
    ticker_symbol = stock["ticker_symbol"]
    for _, article in articles.items():
        revelant_tickers = {_["ticker_symbol"] for _ in article["stock_matches"]}
        if ticker_symbol not in revelant_tickers:
            article_ = {k:v for k, v in article.items() if k in ["doc_id", "title", "url", "summary"]}
            stock["irrelevant_articles"].append(article_)

### Ranking article matches

In [26]:
for _, stock in set_aside_stocks.items():
    stock["relevant_articles"] = sorted(stock["relevant_articles"], 
                                        key=lambda x: x["score"], 
                                        reverse=True)

### Saving to file

In [27]:
with open("../../data/test/user_study/stocks/stocks.v3.json", "w") as fp:
    json.dump(set_aside_stocks, fp, ensure_ascii=False, indent="\t")

In [28]:
len(set_aside_stocks)

75